In [0]:
import json
import re
import cni_connectors.adls_gen1_connector as connector
import pyspark.sql.functions as f
import crawler.functions as cf
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, DoubleType

In [0]:
var_adls_uri = connector.adls_gen1_connect(spark, dbutils, scope="adls_gen2", dynamic_overwrite="dynamic")

fornecedor_dev
dev-execution


In [0]:
#Parametros de entrado do ADF
var_file = {'namespace':'API_InGuru',
'file_folder':'inguro_acidente_trab_sem_obito',
'extension':'csv',
'column_delimiter':',',
'encoding':'UTF-8',
'null_value':''}

#Caminho de Dev
var_dls = {
    "folders":{"landing":"/tmp/dev/uld",
    "error":"/tmp/dev/err",
    "staging":"/tmp/dev/stg",
    "log":"/tmp/dev/log",
    "raw":"/tmp/dev/raw",
    "trusted":"/tmp/dev/trs",
    "business":"/tmp/dev/biz",
    "landing":"/tmp/dev/uld"          }}
#Parametros do ADF (Nativo)
var_adf = {
    "adf_factory_name": "cnibigdatafactory",
    "adf_pipeline_name": "org_raw_acidente_trab_sem_obito",
    "adf_pipeline_run_id": "60ee3485-4a56-4ad1-99ae-666666666",
    "adf_trigger_id": "62bee9e9-acbb-49cc-80f2-666666666",
    "adf_trigger_name": "62bee9e9-acbb-49cc-80f2-66666666",
    "adf_trigger_time": "2020-06-08T01:42:41.5507749Z",
    "adf_trigger_type": "PipelineActivity"
}

In [0]:
#Acesso ao json do CMD
lnd = var_dls['folders']['landing']
raw = var_dls['folders']['raw']

In [0]:
var_source = "{lnd}/{namespace}/{file_folder}/".format(lnd=lnd, namespace=var_file['namespace'], file_folder=var_file['file_folder'])
var_source

Out[27]: '/tmp/dev/uld/API_InGuru/inguro_acidente_trab_sem_obito/'

In [0]:
#Origem e destino
source_lnd = "{adl_path}{lnd}/{namespace}/{file_folder}".format(adl_path=var_adls_uri, lnd=lnd, namespace=var_file["namespace"],file_folder=var_file["file_folder"])
source_lnd

Out[28]: '[REDACTED]/tmp/dev/uld/API_InGuru/inguro_acidente_trab_sem_obito'

In [0]:
azure_path = 'abfss://datalake@cnibigdatadlsgen2.dfs.core.windows.net'
outputh_path = '{uri}/uds/uniepro/API_InGuru/inguro_vig_acidente_trab'.format(uri=azure_path)

def API_INGURU(url, per_page, start_date, end_date):
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
    'accept':'application/json',
    'authorization': '6f64d6a047fad369d35c3806f8f8e0560475075a432585973f91caae35b4ad74'
    } 

    params = {
        'per_page': per_page,
        'start_date': start_date,
        'end_date': end_date,
        'sort': '1'
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [0]:
def divide_data(data_inicial, data_final):
    formato_data = '%d/%m/%Y %H:%M'

    data_inicial = datetime.strptime(data_inicial, formato_data)
    data_final = datetime.strptime(data_final, formato_data)
    intervalo = data_final - data_inicial
    data_final = data_inicial + intervalo // 2
    data_final = data_final.replace(hour=23, minute=59, second=59) # define o final como o último segundo do minuto
    data_final = data_final.strftime(formato_data)
    data_inicial = data_inicial.strftime(formato_data)
    return (data_inicial, data_final)

In [0]:
def load_big_df(path):
  schema = StructType([
    StructField("id", LongType(), nullable=True),
    StructField("domain", StringType(), nullable=True),
    StructField("title", StringType(), nullable=True),
    StructField("subtitle", StringType(), nullable=True),
    StructField("author", StringType(), nullable=True),
    StructField("content", StringType(), nullable=True),
    StructField("url", StringType(), nullable=True),
    StructField("source", StringType(), nullable=True),
    StructField("source_country", StringType(), nullable=True),
    StructField("source_state", StringType(), nullable=True),
    StructField("crawled_date", StringType(), nullable=True),
    StructField("published_date", StringType(), nullable=True),
    StructField("dh_insertion_raw", StringType(), nullable=True)
  ])

  df = spark.read.schema(schema) \
             .option("mergeSchema", "true") \
             .option("timestampFormat", "INT96") \
             .parquet(path) \
             .withColumn("dh_insertion_raw", f.col("dh_insertion_raw").cast("timestamp"))\
             .withColumn("crawled_date", f.col("crawled_date").cast("timestamp"))
  
  return df

In [0]:
def get_news(data_inicial, data_final, id):
    url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

    array_dfs = []

    getted_news = 0
    
    total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']

    
    if not total_news:
      parcial_data_inicial = data_inicial
      parcial_data_final = data_final
      dados = {
        'author': np.nan,
        'content': np.nan,
        'crawled_date': np.nan,
        'domain': np.nan,
        'id': np.nan,
        'published_date': np.nan,
        'source': np.nan,
        'source_country': np.nan,
        'source_state': np.nan,
        'subtitle': np.nan,
        'title': np.nan,
        'url': np.nan,
        'dh_insertion_raw': np.nan
      }
      fail_df = pd.DataFrame(dados, index=[0])
      array_dfs.append(fail_df)
      parcial_data_inicial = parcial_data_final
      parcial_data_final = data_final
    else:
      
      print('Total de notícias: ', total_news)

      parcial_data_inicial = data_inicial
      parcial_data_final = data_final

      while getted_news < total_news:

          parcial_news = total_news

          while parcial_news > 10000:
              print('Parcial de notícias: ', parcial_news)
              parcial_data_inicial, parcial_data_final = divide_data(parcial_data_inicial, parcial_data_final)
              parcial_news = API_INGURU(url, 1, parcial_data_inicial, parcial_data_final)['pagination']['total']

          print("fazendo requisicao")
          print('Data inicial: ', parcial_data_inicial)
          print('Data final: ', parcial_data_final)
          news = API_INGURU(url, parcial_news, parcial_data_inicial, parcial_data_final)['data']

          #CRIA UM DATAFRAME COM AS NOTICIAS
          noticias = pd.DataFrame(news)
          #ADICIONA NUM ARRAY DE DATAFRAME
          array_dfs.append(noticias)

          #corrige as datas

          parcial_data_inicial = parcial_data_final
          parcial_data_final = data_final

          getted_news += parcial_news
          

    return pd.concat(array_dfs)

### Ingestão de anos anteriores

Teve de ser criado pois a API não carrega todos os anos por alguma limitação.

PRECISA SER RODADO APENAS NA PRIMEIRA VEZ

In [0]:
def insere_news(data_inicial, data_final, id, path):
  url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

  array_dfs = []

  getted_news = 0
    
  total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']
    
  if total_news:
      
#     print('Total de notícias: ', total_news)

    parcial_data_inicial = data_inicial
    parcial_data_final = data_final

    while getted_news < total_news:

      parcial_news = total_news

      while parcial_news > 10000:
#         print('Parcial de notícias: ', parcial_news)
        parcial_data_inicial, parcial_data_final = divide_data(parcial_data_inicial, parcial_data_final)
        parcial_news = API_INGURU(url, 1, parcial_data_inicial, parcial_data_final)['pagination']['total']

#       print("fazendo requisicao")
#       print('Data inicial: ', parcial_data_inicial)
#       print('Data final: ', parcial_data_final)
      news = API_INGURU(url, parcial_news, parcial_data_inicial, parcial_data_final)['data']

      #CRIA UM DATAFRAME COM AS NOTICIAS
      noticias = pd.DataFrame(news)

      #corrige as datas
      parcial_data_inicial = parcial_data_final
      parcial_data_final = data_final

      getted_news += parcial_news

      dh_insertion_raw = var_adf["adf_trigger_time"].split(".")[0]

      schema = StructType([
        StructField("id", LongType(), nullable=True),
        StructField("domain", StringType(), nullable=True),
        StructField("title", StringType(), nullable=True),
        StructField("subtitle", StringType(), nullable=True),
        StructField("author", StringType(), nullable=True),
        StructField("content", StringType(), nullable=True),
        StructField("url", StringType(), nullable=True),
        StructField("source", StringType(), nullable=True),
        StructField("source_country", StringType(), nullable=True),
        StructField("source_state", StringType(), nullable=True),
        StructField("crawled_date", StringType(), nullable=True),
        StructField("published_date", StringType(), nullable=True)
      ])

      saveDF = spark.createDataFrame(noticias, schema)
      saveDF = saveDF.withColumn("dh_insertion_raw", f.lit(dh_insertion_raw).cast("string"))
      saveDF.write.parquet(path, "append")
      print("Total:", total_news)
      print("Inseridas:", getted_news)
    
  else:
    print("Sem dados")


In [0]:
graph_ids = []
graph_names = []
graph_dfs = []

nodeID = '926' #Declaração do ID

url_nodeID = 'https://app.inguru.me/api/v1/taxonomies/nodes/'+str(nodeID)
IDs = API_INGURU(url = url_nodeID, per_page=1, start_date='', end_date='')['data'][0]
graph_ids.append(IDs['id'])
graph_names.append(IDs['name'].replace(" ", "_"))

#Passa por cada "filho" no nó principal, caso não tenha nenhum, nada acontece.
for node_children in IDs['children']:
  graph_ids.append(node_children['id'])
  graph_names.append(node_children['name'].replace(" ", "_"))

data_inicial = '01/01/2015 00:00'
data_final = datetime.now().strftime("%d/%m/%Y %H:%M")

for i, id in enumerate(graph_ids):
  path = source_lnd + "_" + str(graph_names[i])
  insere_news(data_inicial, data_final, id, path)


Total: 696086
Inseridas: 537
Total: 696086
Inseridas: 8733
Total: 696086
Inseridas: 17920
Total: 696086
Inseridas: 22655
Total: 696086
Inseridas: 23721
Total: 696086
Inseridas: 33015
Total: 696086
Inseridas: 39495
Total: 696086
Inseridas: 43091
Total: 696086
Inseridas: 48819
Total: 696086
Inseridas: 53625
Total: 696086
Inseridas: 58873
Total: 696086
Inseridas: 68117
Total: 696086
Inseridas: 73709
Total: 696086
Inseridas: 79346
Total: 696086
Inseridas: 86106
Total: 696086
Inseridas: 91100
Total: 696086
Inseridas: 96653
Total: 696086
Inseridas: 102298
Total: 696086
Inseridas: 107228
Total: 696086
Inseridas: 112855
Total: 696086
Inseridas: 117813
Total: 696086
Inseridas: 124897
Total: 696086
Inseridas: 134153
Total: 696086
Inseridas: 143895
Total: 696086
Inseridas: 150141
Total: 696086
Inseridas: 158429
Total: 696086
Inseridas: 165030
Total: 696086
Inseridas: 173475
Total: 696086
Inseridas: 183182
Total: 696086
Inseridas: 192954
Total: 696086
Inseridas: 201457
Total: 696086
Inseridas: 209

In [0]:
# CARREGAMENTO DAS TABELAS PARA VER SE FORAM SALVAS CORRETAMENTE
test_dfs = []
for i, df in enumerate(graph_dfs):
  path = source_lnd + "_" + str(graph_names[i])
  test_dfs.append(load_big_df(path))
  print(path)

In [0]:
print(test_dfs[1].count())

282

In [0]:
#ZERAR A BASE

dh_insertion_raw = var_adf["adf_trigger_time"].split(".")[0]

data = {
  'id': 0,
  'domain': 'Não informado',
  'title': 'Não informado',
  'subtitle': 'Não informado',
  'author': 'Não informado',
  'content': 'Não informado',
  'url': 'Não informado',
  'source': 'Não informado',
  'source_country': 'Não informado',
  'source_state': 'Não informado',
  'crawled_date': '1900-01-01 00:00:00',
  'published_date': '1900-01-01 00:00:00',
  'dh_insertion_raw': '2020-06-08T01:42:41.000+0000'
}
schema = StructType([
  StructField("id", LongType(), nullable=True),
  StructField("domain", StringType(), nullable=True),
  StructField("title", StringType(), nullable=True),
  StructField("subtitle", StringType(), nullable=True),
  StructField("author", StringType(), nullable=True),
  StructField("content", StringType(), nullable=True),
  StructField("url", StringType(), nullable=True),
  StructField("source", StringType(), nullable=True),
  StructField("source_country", StringType(), nullable=True),
  StructField("source_state", StringType(), nullable=True),
  StructField("crawled_date", StringType(), nullable=True),
  StructField("published_date", StringType(), nullable=True),
  StructField("dh_insertion_raw", StringType(), nullable=True)
  ])

data['id'] = int(data['id'])

# dfff = pd.DataFrame(data, index=[0])
spark_dfff = spark.createDataFrame([tuple(data.values())], schema=schema)
for i, df in enumerate(graph_ids):
  print(graph_names[i])
  path = source_lnd + "_" + str(graph_names[i])
  spark_dfff.write.parquet(path, mode="overwrite")
  

Nodo_Raiz
CASOS
ACIDENTES_DE_TRABALHO


In [0]:
display(spark.read.parquet(path))

id,domain,title,subtitle,author,content,url,source,source_country,source_state,crawled_date,published_date,dh_insertion_raw
0,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,1900-01-01 00:00:00,1900-01-01 00:00:00,2020-06-08T01:42:41.000+0000
